# Manual Measurement Plant AutoGenerate Configurations from CSV

This notebook will focus on the Input_helper configuration generation for us with the plant integration within Home Assistant.

Date of Project: July 24 2022
Software and Devices Used: 
- Home Assistant 2022.7
- Awair Element AQI Sensor
- Hue Motion Sensor + Hue Bridge

Further notebooks will be created to document the other portions of this project which will be available on my Github repositoriy located here.

https://github.com/netmanchris/HassPlants



# Auto Generation of Manual Input Plant integrations for Home Assistant

Like most things, what was easy at low volume can start to show challenges when reaching certain scales. 

This project is intended to help automate the creation of my 
home assistant based Plant monitoring setup. Although I prefer to use the Xioami Miflora sensors who's initial setup can be seen documented in this [project](./README.MD).



## WHY?

### Why part 1?

I killed plants. I'd like my plants to not die.

### Why part 2?

Why not? Home Automation is supposed to help us and make our lives easier. 

### Why part 3?

MifLora Sensors are kinda awesome, but at scale they can start to be very expensive. When I started this project last year, they could be found as low as eleven dollars a piece. At the time of this writting, they are showing on AliExpress at twenty-six average and somewhere around fifty on Amazon Canada.  Too rich for my blood.

### Simple Solution

For users not wanting to invest in the ESP32 + Miflora sensors. You can leverage a combination of a traditional plant sensor to take and track manual readings in combination of existing temperature and light sensors that you likely might already have in your homes.





# CSV File Formating and Plant integration Generation

There are currently three files which are required as data inputs for this project

- plants_worksheet.csv is the file where the plants to include in the generation of the input_helpers will be stored
- plants_attribytes.csv
- room_sensors.csv





## Plants Worksheet CSV

The Plants worksheet is a CSV which contains all of the information required to generate the ESP32 configurations, plant integration configurations, as well as the dashboards for both desktop/tablet and mobile formats.

The plant worksheet can be located here.

Each row of this worksheet represents a single plant with the following attributes\

- name: Unique Value. One per plant. type: String
- plant_type: One per plant. Must match values in the *plant_attributes.csv* worksheet type: String
- room: Grouping mechanism. Plants with the same value will be automatically grouped together.
- sensor: Unique Value. One per sensor. type: Integer
- sensor_number: Unique Value. One per sensor. This will be appended in the ESP config as the unique portion of the auto-generated sensor entity.  type: String
- sensor_mac: the 12 digit mac-address in the format XX:XX:XX:XX:XX:XX example C4:7C:8D:6D:8D:F6 type: String
- esp_parent: {FUTURE USE} Intention is to use this to create a permanent binding between a specific plant sensor and a specific ESP32 device to prevent naming conflicts when dealing with larger rooms ( > 13 plants per room ) type: String 
- esp_current: Unused. Documentation purposes only. type: String
- room_picture: {FUTURE USE} Intention is to use this to bind specific plants to specific room pictures for building - out dashboards that require multiple camera angles. type: String
- top: Numerical value used a the X coodinate for plant placement for the room picture generation. type: Integer
- left: Numerical value used a the Y coodinate for plant placement for the room picture generation. type: Integer
- flash_light: full name of the script entitiy used to flash the lights where a specific plant is located. type: String








## Plants Attributes CSV

The Plants worksheet is a CSV which contains all of the information required to generate the ESP32 configurations, plant integration configurations, as well as the dashboards for both desktop/tablet and mobile formats.

The plant worksheet can be located [here](./data/plant_attrributes.csv). Further documentation on the exact values below can be found on the Home Assistant Plant integration page [here](https://www.home-assistant.io/integrations/plant/)

Each row of this worksheet represents a single plant with the following attributes


- plant: Unique Value. One per plant species. type: String
- species: Unique value. One per plant species. type: String
- min_moisture: minimum moisture value reported by linked sensor where Home Assistant should trigger an alarm condition. Type: Integer between 1 and 100
- max_moisture: maximum moisture value reported by linked sensor where Home Assistant should trigger an alarm condition. Type: Integer between 1 and 100
- min_conductivity: minimum soil fertilization level value reported by linked sensor where Home Assistant should trigger an alarm condition. Type: Integer
- max_conductivity: maximum soil fertilization level value reported by linked sensor where Home Assistant should trigger an alarm condition. Type: Integer
- min_temperature: minimum temperature value reported by linked sensor where Home Assistant should trigger an alarm condition. Type: Integer
- max_temperature: maximum temperature value reported by linked sensor where Home Assistant should trigger an alarm condition. Type: Integer
- min_brightness: minimum brightness value reported by linked sensor where Home Assistant should trigger an alarm condition. Type: Integer
- max_brightness: maximum brightness value value reported by linked sensor where Home Assistant should trigger an alarm condition. Type: Integer
- check_days: time interval (in days) used when checking min_brightness.

## Room Sensors CSV


The Room Sensors file is a CSV which contains some of the information required to generate the manual sensor plant integration configurations.

The room sensor worksheet can be located [here](./data/room_sensors.csv). 

Each row of this worksheet represents a single room with the following attributes

- room_name: Room name which must match the room name in the other spreadsheets. Type: String
- room: Room name which must match the room name in the other spreadsheets. Type: String
- temp_sensor: The full entity name of a device physically located in the same room as a plant which can be used as a proxy temperature sensor. Type: String
- lux_sensor: The full entity name of a device physically located in the same room as a plant which can be used as a proxy lux sensor. Type: String



# Manual Sensor  Plant Integration Jinja Template

The Goal of this project will be to do the following

- Identify and Filter out only the plants in the plants_worksheet.csv file which are listed as having manual sensors. 
- Write a set of files to disk for Plant integration Definitions. One file per room which should contain all plants in that room. 

Bottom line; a complete plant integration entity will be created for each manual sensor in the spreadsheet and placed in a file labeled with the title of the room where that plant is located.  

The template is split into 3 sections

1. Bind Input Helpers to this Plant - This will automatically generate the plant definition including the appropriate binding to the right moisture and soil_conductivity input_helper entities for the specific plant.

2. Bind Sensors to this Plant - This will automatically generate the plant definition including the appropriate binding to the right temperature and brightness sensor entities for the specific plant.

3. Fill in Plant Species Specific Values - This should automatically generate the plant definition included the appropriate values based on the plants species from the plants_attributes.csv file above. 

```
{% for plant in plants %}{% if plant.sensor_model == 'manual' %}
{{plant.name}}:
    sensors:
      moisture: input_number.{{plant.name|lower}}_moisture
      conductivity: input_number.{{plant.name|lower}}_soil_conductivity{% for sensor in manual_sensors %}{% if sensor.room == plant.room %}
      temperature: {{ sensor.temp_sensor }}
      brightness: {{ sensor.lux_sensor }} {% endif %}{% endfor %}{% for plant_type in plant_types%}{% if plant.plant_type == plant_type.species%}
    min_moisture: {{plant_type.min_moisture}}
    max_moisture: {{plant_type.max_moisture}}
    min_conductivity: {{plant_type.min_conductivity}}
    max_conductivity: {{plant_type.max_conductivity}}
    min_temperature: {{plant_type.min_temperature}}
    max_temperature: {{plant_type.max_temperature}}
    #min_brightness: {{plant_type.min_brightness}}
    max_brightness: {{plant_type.max_brightness}}{% endif%}{% endfor %}
    check_days: 3

    {% endif %}{% endfor %}
```





# Generating the Configuration Files

In this section, we'll go through the python code used to generate the actual configurations





In [13]:
import pandas as pd
from jinja2 import Template
import os

# Load Required Data Files CSV

In this section, we will load the plants CSV file which will be used to create the ESPHome configurations to bind the Miflora BLE Plant sensors to a specific ESP32. 

There are two files needed here.

1. plants_worksheet.csv which includes the various plants, sensors, room locations, corresponding script to flash lights.

2. plants_attributes.csv which includes the specific min/max values for the various sensors to be used with the plant integration

3. room_sensors.csv which includes the home assistant sensors for temperature and brightness for the room where the plant is located. 



In [18]:
#plants_df = pd.read_csv('data/plants_worksheet.csv', index_col=0)
plants_df = pd.read_csv('data/plants_worksheet_sample.csv', index_col=0)
plant_attribs = pd.read_csv('data/plant_attrributes.csv', index_col=0).to_dict('records')
manual_sensors = pd.read_csv('data/room_sensors.csv', index_col=0).to_dict('records')



## Create Rooms List

In this step, we will create a list of the room names defined in the CSV file. Defining the room names logically is important in that ESP32 configurations, Plant entity definitions, and dashboards for each individual room will be automatically generated through this code. Spending some time thinking about how you want to organize your plants dashboards is a very, very good idea.

Note: I've got logic in the jinja2 files that will check to see whether the element in the room list is of type "string" which will allow sensors unassigned to rooms to be ignored. 

TODO - CHECK TO MAKE SURE THIS IS TRUE FOR ALL TEMPLATES

In [19]:
rooms_list = list(plants_df.room.unique())
rooms_list

['boys_bedroom',
 'dining_room',
 'downstairs_bathroom',
 'games_room',
 'home_office',
 'kitchen',
 'living_room',
 'master_bedroom',
 nan]

## Loading the Plants Integration Jinja2 Templates

In this step, we load the plants specific templates from the templates folder and load it as a jinja2 template.




In [23]:
with open('templates/room_plants.j2') as f:
    room_template = f.read()
j2_plantroom_template = Template(room_template)

## Rendering Plant Configurations

We will now render the Plant Configurations where one file will be generated for each room




Note: As a prerequisite, the directory structure has to preexist to be able to properly save the file configurations to disk. 

Make sure that the folder "plants" exists under the output folder.







In [24]:
# Includes the manual plants
# Includes the LilyGo plants

for room in rooms_list:
    if type(room) is str:  
        room = [room]
        room_df = plants_df[plants_df.room.isin(room)]
        #print (len(room_df))
        if len(room_df) > 0:
            #print (room)
            #room_df = room_df[room_df.sensor_model.eq('Miflora')]
            room_dict = room_df.to_dict('records')
            #print (len(room_dict))

            # OUTPUT A SINGLE FILE FOR EACH ROOM WITH ALL THE MiFLORA PLANTS INTEGRATION YAML WITH THE PROPER MIFLORA SENSOR NAMES
            with open('output/plants/'+room_dict[0]['room']+'.txt', 'w') as f: 
                f.write(j2_plantroom_template.render(plants=room_dict, plant_types=plant_attribs, manual_sensors = manual_sensors))

# Sample Output

The code above should generate a single file per room which contains all of the required plants.

The following is the a sample of the file labeled "boys_bedroom.txt".

As you can see, the plant definition is automatically using the sensor name which matches that auto-generated in the [Auto Generation of input_helper Configurations for manual tracking of plants in home assisant](./AutoGeneration of Input_Helpers for Plants without Sensors.ipynb) section of this project.

You can verify that the moisture and conductivity sensors are the input_numbers generated in the other project, but the temperature and brightness sensors are values derived for plants in the boys_bedroom room from the room_sensors.csv file.

```
BBD_Snake_1:
    sensors:
      moisture: input_number.bbd_snake_1_moisture
      conductivity: input_number.bbd_snake_1_soil_conductivity
      temperature: sensor.awair_bbd_temp
      brightness: sensor.hue_motion_sensor_1_illuminance_4 
    min_moisture: 7
    max_moisture: 50
    min_conductivity: 300
    max_conductivity: 1000
    min_temperature: 7
    max_temperature: 32
    #min_brightness: 1500
    max_brightness: 60000
    check_days: 3

```

The configuration should be cut and paste into a file labeled %room_name%.yaml in the appropriate folder

```
config
   ->plants
```
       
This should also require adding a line to the configuration.yaml file to make sure that the structure is included in the processing of the overall configuration.


```plant: !include_dir_merge_named plants```

![image](./images/config_plants_yaml.png)


Once you reload your configuration or restart your home assistant instance, you should be able to verify that the plants are shown as follows


Navigate to **Settings > Devices & Services > entities** to validate that the plants were created as desired.

![image](./images/hass_plant_entities.png)

By click on a specific entity you can validate the current status and configuration 

![image](./images/plant_entity_3.png)

By clickig on the attributes link, you can also validate the specific sensors that any individual plant. Notice that in the attributes you can see that the input_helpers have been assigned to the moisture and soil conductivity attributes, but that the Awair and Hue sensors have been used for the temperature and brightness values.

![image](./images/plant_entity_4.png)
